In [ ]:
websites = [
    # "https://morth.nic.in/print/OMs-Circular-Other-Notification",
    # "https://morth.nic.in/print/establishment",
    # "https://morth.nic.in/establishment-archive",
    # "https://morth.nic.in/nh-entrustment",
    # "https://morth.nic.in/Motor-Vehicle-Legislation",
    # "https://morth.nic.in/nh-declaration",

    # "https://incometaxindia.gov.in/pages/communications/circulars.aspx",
    # "https://irrigation.telangana.gov.in/icad/getCirculars?pagenum=1",
    # "https://doe.gov.in/orders-circulars",
    # "https://rajbhasha.gov.in/en/orders-and-circulars",
    # "https://rajbhasha.gov.in/en/orders-and-circulars/orders-and-circulars-archive",
    # "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListing=yes&sid=1&ssid=7&smid=0",
    # "https://www.sebi.gov.in/sebiweb/home/HomeAction.do?doListingCirArchive=yes&sid=1&ssid=7&smid=0",
    # "https://www.aicte-india.org/bulletins/circulars",
    # "https://www.rbi.org.in/Scripts/BS_CircularIndexDisplay.aspx",
    # "https://www.esic.gov.in/circulars",
    # "https://prasarbharati.gov.in/circulars/",
    # "https://dot.gov.in/all-circulars-archives",
    # "https://dot.gov.in/all-circulars?tid=All",
    # "https://indianrailways.gov.in/railwayboard/view_section.jsp?id=0,1,304,366,390",
    # "https://gstcouncil.gov.in/circulars",
    # "https://main.sci.gov.in/notices-circulars",
    # "http://exam.unipune.ac.in/Pages/Circulars.html",
    # "https://www.civilaviation.gov.in/en/documents/circulars",
    # "https://labour.gov.in/child-labour-new-circulars",
    # "https://financialservices.gov.in/circulars-3",
    # "https://main.mohfw.gov.in/Circulars",
    # "https://pmssy-mohfw.nic.in/index4.php?lang=1&level=0&linkid=33&lid=37",
    # "https://www.moh.gov.om/en/web/dgpadc/-11",
    # "https://www.ugc.gov.in/ugc_circular.aspx",
    # "https://uidai.gov.in/en/media-resources/uidai-documents/circulars-memorandums-notification-01.html",
    # "https://dopt.gov.in/notifications/orders",
    # "https://doptcirculars.nic.in/Default.aspx?URL=e0joY8Gw069R",
    # "https://www.indiapost.gov.in/VAS/Pages/AboutUs/Circular_2021.aspx",
    # "https://www.dgshipping.gov.in/Content/DGSCirculars.aspx",
    # "https://services.delhi.gov.in/circulars",
    # "https://rajbhasha.gov.in/en/Circular-and-orders",
    # "https://cbic-gst.gov.in/hindi/circulars.html",
    # "https://vuao.nic.in/DOPT_VUAO_Form1.aspx",
    "https://urban.rajasthan.gov.in/content/raj/udh/udh-department/en/new-orders-and-circulars.html",  #don't work
    "https://dopt.gov.in/whats-new",
    "https://dopt.gov.in/whats-new/whats-new-archive",  # till here
    # # "https://finance.mp.gov.in/circular",
    # "https://uppolice.gov.in/article/en/department-circulars",
    # "https://finance.cg.gov.in/vitt_nirdesh/circulars.asp",    #giving 0 output
    # "https://state.bihar.gov.in/finance/SectionInformation.html?editForm&rowId=2993",
    # "https://icar.org.in/circulars",
    "https://dopjharkhand.gov.in/public/cms.aspx?id=20",  # working
    # "https://www.epfindia.gov.in/site_en/circulars.php",
    "https://www.uppcl.org/uppcl/en/circulars?cd=NgA1AA%3D%3D",
    "https://cag.gov.in/ag/manipur/en/ae-circulars-office-orders",
    "https://mpeda.gov.in/?page_id=1127",
    "https://jrhms.jharkhand.gov.in/circulars",
    # "https://mphc.gov.in/circulars-orders",
    "https://weightnmeasures.delhi.gov.in/weightnmeasures/gazette-notifications-and-circulars",
    # "https://cga.nic.in/Circular/Published/list.aspx",  # not iitb
    # "https://www.pci.nic.in/PolicyCircular.html",
    # "http://119.226.139.196/ddaweb/circulars.aspx",
    # "https://upite.gov.in/UPLC/DynamicPages/GO.aspx",    # no pdf in this link
    # "https://www.ceodelhi.gov.in/CircularsOrdersNotifications.aspx",
    # "https://knit.ac.in/en/circulars",
    "http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp",
    # "https://urban.rajasthan.gov.in/content/raj/udh/udh-department/en/orders-circulars.html"   # Not Working
]

# To access the list, you can iterate over it using a for loop or access individual elements by index.


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin, urlparse
import re
from requests.adapters import HTTPAdapter, Retry

# all together

In [ ]:

def is_pdf_link(link):
    return link.lower().endswith('.pdf')

def is_redirect_to_pdf(link):
    try:
        response = requests.head(link, allow_redirects=True)
        return response.headers.get('content-type') == 'application/pdf'
    except requests.RequestException:
        return False

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        # Setting up retry strategy
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            # method_whitelist=["HEAD", "GET", "OPTIONS"],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        # Fetch the page with a timeout
        response = http.get(url, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None

# Function to extract PDF links from a webpage
def extract_pdf_links(soup, base_url):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if is_pdf_link(link['href']) or is_redirect_to_pdf(link['href']):
          print(urljoin(base_url, link['href']))
          pdf_links.append(urljoin(base_url, link['href']))
      except:
        print('Problem IN the PARSING')
        pass
    # pdf_links = [link['href'] for link in links if link['href'].endswith('.pdf')]
    return pdf_links

# Function to find and follow nested links
def find_and_follow_links(soup, base_url, visited_urls):
    nested_links = []
    print(base_url)
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('http') or href.startswith('www'):
            full_url = href
        else:
            full_url = base_url.rstrip('/') + '/' + href.lstrip('/')

        if full_url not in visited_urls:
            nested_links.append(full_url)
    return nested_links

def find_next_page(soup):
    next_page = soup.find('a', string=re.compile(r'next\s*>?', re.IGNORECASE))
    if next_page:
        return next_page.get('href')
    return None

# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup, url))

    next_page = find_next_page(soup)
    print(next_page)
    base_url = urlparse(url)
    base_url = f"{base_url.scheme}://{base_url.netloc}"

    if next_page:
      while(True):
        if not (next_page.startswith('http') or next_page.startswith('www')):
          next_page = base_url+'/'+next_page
        print('next_page :', next_page)
        visited_urls.add(next_page)
        soup = fetch_page(next_page)
        pdf_links.update(extract_pdf_links(soup, next_page))

        next_page = find_next_page(soup)
        if not next_page:
            break


    elif not next_page:
      nested_links = find_and_follow_links(soup, url, visited_urls)
      # print('nested Links ;', nested_links)

      for nested_link in list(set(nested_links)):
          print('nested_page :', nested_link)
          soup = fetch_page(nested_link)
          if not soup or soup is None:
              print('Soup Not GOOD')
              continue

          pdf_links.update(extract_pdf_links(soup, nested_link))

    return pdf_links


all_pdfs = set()

for website in websites:
    print(f"Scraping PDFs from {website}", end=" ; ")
    pdf_links = extract_pdfs_from_website(website)
    print("found :", len(pdf_links))
    all_pdfs.update(pdf_links)
    time.sleep(1)  # To avoid overloading the server

# Create a DataFrame
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)



Scraping PDFs from https://mpeda.gov.in/?page_id=1127 ; 

Failed to connect to https://mpeda.gov.in/?page_id=1127.
found : 0
Scraping PDFs from https://jrhms.jharkhand.gov.in/circulars ; 

Failed to connect to https://jrhms.jharkhand.gov.in/circulars.
found : 0
Scraping PDFs from https://mphc.gov.in/circulars-orders ; https://mphc.gov.in/PDF/web_pdf/CI/circular no 16 regarding efiling mandatory from 1 st August 2024 at High Court jabalpur and bench indore and gwalior.pdf
https://mphc.gov.in/PDF/web_pdf/CI/B-1816 Dt. 25.05.2024.pdf
https://mphc.gov.in/PDF/web_pdf/CI/Regarding Designation of Senior Advocates..pdf
https://mphc.gov.in/PDF/web_pdf/CI/Memo No. C-4002. Dtd. 14-05-2024.pdf
https://mphc.gov.in/PDF/web_pdf/CI/img20240510_10482214.pdf
https://mphc.gov.in/PDF/web_pdf/CI/C-3928, dt 10.05.24.pdf
https://mphc.gov.in/PDF/web_pdf/CI/NOTIFICATION NO. A 3278 DATED 29-04-2024 (CHAIRMAN OF THE HIGH COURT LEGAL SERVICES COMMITTEE, BENCH AT GWALIOR).pdf
https://mphc.gov.in/PDF/web_pdf/CI/Notice No. A-3208 Jabalpur dated 26.04.2024 ( Regarding invitation of Application of Class-IV employees for the post of Jr. Judicial Assistant).pdf
https://mphc.gov.in/PDF/web_pdf/CI/A3093_Reg

nested_page : https://mphc.gov.in/PDF/web_pdf/CI/Notification No. C-638 Dated 18.02.2021.pdf


nested_page : https://mphc.gov.in/PDF/web_pdf/CI/b_3268 dtd 28-08-2020 (Unit) Reg. consideration on units for compromise due to replacement of schedule under Section 320 of Cr.P.C..pdf


...

nested_page : https://mphc.gov.in/circulars-orders/styleswitcher/switch/mphc_theme/custom/black?destination=node/5
HTTP error occurred: 404 Client Error: Not Found for url: https://mphc.gov.in/circulars-orders/styleswitcher/switch/mphc_theme/custom/black?destination=node/5 - https://mphc.gov.in/circulars-orders/styleswitcher/switch/mphc_theme/custom/black?destination=node/5
Soup Not GOOD
nested_page : https://mphc.gov.in/PDF/web_pdf/CI/Armed Force Tribunal, Regional Bench, Jabalpur Retired Servants.pdf


nested_page : https://mphc.gov.in/PDF/web_pdf/CI/Circular No. A-3120 Dated 11.12.2020 for Subordinate Court.pdf


nested_page : https://mphc.gov.in/PDF/web_pdf/CI/a-530-240317.PDF


nested_page : https://mphc.gov.in/PDF/web_pdf/CI/Endo. No. B-8148 Jabalpur, dated 07-11-2023 (vidhansabha election 2023).pdf


KeyboardInterrupt: 

# it.delhigovt

In [ ]:
link = "http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp"
import requests
from bs4 import BeautifulSoup
import pandas as pd


# URL of the first page
base_url = "http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page="
page_number = 1
max_pages = 3
pdf_url = 'http://it.delhigovt.nic.in'


def extract_pdf_links(url, base):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    pdf_links = [base+link.get('href') for link in links if link.get('href') and (link.get('href').endswith('.pdf') or link.get('href').endswith('.PDF'))]
    print(pdf_links[0])
    return pdf_links

all_pdf_links = []
while True:
    url = f"{base_url}{page_number}"
    print(f"Fetching {url}")

    pdf_links = extract_pdf_links(url, pdf_url)
    all_pdf_links.extend(pdf_links)
    response = requests.get(url)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.content, 'html.parser')
    if page_number == max_pages:
        break

    page_number += 1

Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page=1
http://it.delhigovt.nic.in/writereaddata/Cir20246172.pdf


<ipython-input-13-05734f97c546>:19: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  next_page = soup.find('a', text='[Next >>]')  # Adjust based on your pagination link text


Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page=2
http://it.delhigovt.nic.in/writereaddata/Cir2024525828.pdf
Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page=3
http://it.delhigovt.nic.in/writereaddata/Cir2024261427. 172 dated 10.05.2024.pdf
Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page=4
http://it.delhigovt.nic.in/writereaddata/Cir2024525814.pdf
Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page=5
http://it.delhigovt.nic.in/writereaddata/Cir202455687.pdf
Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page=6
http://it.delhigovt.nic.in/writereaddata/Cir202455684.pdf
Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page=7
http://it.delhigovt.nic.in/writereaddata/Cir202455686.pdf
Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_result.asp?page=8
http://it.delhigovt.nic.in/writereaddata/Cir2024525797.pdf
Fetching http://it.delhigovt.nic.in/pis/noc/cir_search_re

KeyboardInterrupt: 

In [ ]:

# Create a DataFrame
df = pd.DataFrame(all_pdf_links, columns=['PDF Links'])

# Save the DataFrame to a CSV file (optional)
df.to_csv('pdf_links.csv', index=False)

# Display the DataFrame
print(df)

                                              PDF Links
0     http://it.delhigovt.nic.in/writereaddata/Cir20...
1     http://it.delhigovt.nic.in/writereaddata/Cir20...
2     http://it.delhigovt.nic.in/writereaddata/Cir20...
3     http://it.delhigovt.nic.in/writereaddata/Cir20...
4     http://it.delhigovt.nic.in/writereaddata/Cir20...
...                                                 ...
1589  http://it.delhigovt.nic.in/writereaddata/Cir20...
1590  http://it.delhigovt.nic.in/writereaddata/Cir20...
1591  http://it.delhigovt.nic.in/writereaddata/Cir20...
1592  http://it.delhigovt.nic.in/writereaddata/Cir20...
1593  http://it.delhigovt.nic.in/writereaddata/Cir20...

[1594 rows x 1 columns]


# finance

In [ ]:
link = "https://finance.cg.gov.in/vitt_nirdesh/circulars.asp"
base_link = "https://finance.cg.gov.in/vitt_nirdesh/"

In [ ]:
def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup, base_url):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if link['href'].endswith('.pdf'):
          print(urljoin(base_url, link['href']))
          pdf_links.append(urljoin(base_url, link['href']))
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


def find_and_follow_links(soup, base_url, visited_urls):
    nested_links = []
    print(base_url)
    for link in soup.find_all('a', href=True):
        href = link['href']
        print(href)
        if href.startswith('http') or href.startswith('www'):
            full_url = href
        else:
            full_url = base_link + href

        if full_url not in visited_urls:
            nested_links.append(full_url)
    return nested_links

def find_next_page(soup):
    next_page = soup.find('a', string=re.compile(r'next\s*>?', re.IGNORECASE))
    if next_page:
        return next_page.get('href')
    return None

# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup, url))

    next_page = find_next_page(soup)
    print(next_page)
    base_url = urlparse(url)
    base_url = f"{base_url.scheme}://{base_url.netloc}"

    if next_page:
      while(True):
        if not (next_page.startswith('http') or next_page.startswith('www')):
          next_page = base_url+'/'+next_page
        print('next_page :', next_page)
        visited_urls.add(next_page)
        soup = fetch_page(next_page)
        pdf_links.update(extract_pdf_links(soup, next_page))

        next_page = find_next_page(soup)
        if not next_page:
            break


    elif not next_page:
      nested_links = find_and_follow_links(soup, url, visited_urls)
      # print('nested Links ;', nested_links)

      for nested_link in list(set(nested_links)):
          print('nested_page :', nested_link)
          soup = fetch_page(nested_link)
          if not soup or soup is None:
              print('Soup Not GOOD')
              pass
          pdf_links.update(extract_pdf_links(soup, nested_link))

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)



None
https://finance.cg.gov.in/vitt_nirdesh/circulars.asp
subject.asp?subject=1
subject.asp?subject=2
subject.asp?subject=3
subject.asp?subject=4
subject.asp?subject=5
subject.asp?subject=6
subject.asp?subject=7
subject.asp?subject=8
subject.asp?subject=9
subject.asp?subject=10
subject.asp?subject=11
subject.asp?subject=12
subject.asp?subject=13
subject.asp?subject=14
subject.asp?subject=15
subject.asp?subject=16
subject.asp?subject=17
subject.asp?subject=18
subject.asp?subject=19
subject.asp?subject=20
subject.asp?subject=21
subject.asp?subject=22
subject.asp?subject=23
subject.asp?subject=3024
subject.asp?subject=2024
subject.asp?subject=2023
subject.asp?subject=2022
subject.asp?subject=2021
subject.asp?subject=2020
subject.asp?subject=2019
subject.asp?subject=2018
subject.asp?subject=2017
subject.asp?subject=2016
subject.asp?subject=2015
subject.asp?subject=2014
subject.asp?subject=2013
subject.asp?subject=2012
subject.asp?subject=2011
subject.asp?subject=2010
subject.asp?subject=20

# ceodelhi

In [ ]:
link = 'https://weightnmeasures.delhi.gov.in/weightnmeasures/gazette-notifications-and-circulars'

In [ ]:
def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup, base_url):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if link['href'].endswith('.pdf') or link['href'].endswith('.PDF'):
          print(urljoin(base_url, link['href']))
          pdf_links.append(urljoin(base_url, link['href']))
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup, url))

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)



Failed to connect to https://mpeda.gov.in/?page_id=1127.
Empty DataFrame
Columns: [PDF Links]
Index: []


# NCERT

In [ ]:
link = "https://ncert.nic.in/sections/vls/circular.php"
base_link = "https://ncert.nic.in/sections/vls/"

In [ ]:
def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if link['href'].endswith('.pdf') or link['href'].endswith('.PDF'):
          print(urljoin(base_link, link['href']))
          pdf_links.append(urljoin(base_link, link['href']))
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup))

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)



https://ncert.nic.in/sections/vls/pdf/IEMs_NCERT.pdf
https://ncert.nic.in/sections/vls/pdf/CircularVigilanceFunction.pdf
https://ncert.nic.in/sections/vls/pdf/CircularHonorarium.pdf
https://ncert.nic.in/sections/vls/pdf/memorandum_11082022.pdf
https://ncert.nic.in/sections/vls/pdf/memorandum_19072022.pdf
https://ncert.nic.in/sections/vls/pdf/memorandum_05012023.pdf
                                           PDF Links
0  https://ncert.nic.in/sections/vls/pdf/Circular...
1  https://ncert.nic.in/sections/vls/pdf/memorand...
2  https://ncert.nic.in/sections/vls/pdf/memorand...
3  https://ncert.nic.in/sections/vls/pdf/IEMs_NCE...
4  https://ncert.nic.in/sections/vls/pdf/memorand...
5  https://ncert.nic.in/sections/vls/pdf/Circular...


In [ ]:
link = "https://ncert.nic.in/archive-academic.php"
base_link = "https://ncert.nic.in/"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if link['href'].endswith('.pdf') or link['href'].endswith('.PDF'):
          print(urljoin(base_link, link['href']))
          pdf_links.append(urljoin(base_link, link['href']))
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup))

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)

https://ncert.nic.in/pdf/leading_the_change.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/Advertisement_No_173_2022-15-03-2024.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/LDE223.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/Status_Different-areas_21-12-2023.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/ResultNotice_22_11_SA.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/ResultNotice_18_11_SA.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/Interview_Notice_Schedule_AssistantProfessorAgriculture_30-10-23.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/Interview_Notice_Schedule_AssistantProfessorAgriculture.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/Postponement_Notice_AssistantProfessorAgriculture.pdf
https://ncert.nic.in/pdf/announcement/vacancies/academicvacancy/Interview_Notice_Schedule_AssistantProfessorMarathi

In [ ]:
link = "https://ncert.nic.in/notices.php?ln=en"
base_link = "https://ncert.nic.in/"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if link['href'].endswith('.pdf') or link['href'].endswith('.PDF'):
          print(urljoin(base_link, link['href']))
          pdf_links.append(urljoin(base_link, link['href']))
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup))

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)

https://ncert.nic.in/pdf/leading_the_change.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/HAC_668.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/Circular_C&W_662_667.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/Final_&_Tentative_list_Type-I-Type-V1027-1032.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/tentativelistJanuary2023_June2023.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/circular_objection090223.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/Final_listType_I-V_20April22.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/circular_objection11222.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/Final_listType_I-V_21July.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/TentativeType_I-V_21april.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/C&W_1467.pdf
https://ncert.nic.in/pdf/announcement/notices/campusWelfare/Obje

# UGC

In [ ]:
link = "https://ugcnet.nta.nic.in/archive/"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if link['href'].endswith('.pdf') or link['href'].endswith('.PDF'):
          print(link['href'])
          pdf_links.append(link['href'])
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup))

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)

https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023010511.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023010511.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023010511.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023010587.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023010587.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023010587.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023011934.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023011934.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023011934.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/01/2023012135.pdf
https://cdnbbsr.s3waas.gov.in/s301eee509

In [ ]:
link = "https://cup.ugc.ac.in/Circular.aspx"
base_link = "https://cup.ugc.ac.in/"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout, verify = False)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if link['href'].endswith('.pdf') or link['href'].endswith('.PDF'):
          print(urljoin(base_link, link['href']))
          pdf_links.append(urljoin(base_link, link['href']))
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup))

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'cup.ugc.ac.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://cup.ugc.ac.in/Docs/LIST_OF_56_CENTRAL_UNIVERSITIES_and_PRINCIPAL_UCMS.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-22-03-2024-12-19-12.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-22-03-2024-12-31-45.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-22-03-2024-12-26-01.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-19-01-2024-02-33-02.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-16-11-2023-09-56-22.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-16-11-2023-12-34-21.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-16-11-2023-10-22-31.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-16-11-2023-10-07-04.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-16-11-2023-12-20-32.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-16-11-2023-11-46-17.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-16-11-2023-12-27-15.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-16-11-2023-12-14-34.pdf
https://cup.ugc.ac.in/notice_upload/CUP-Notice-09-11-2023-09-58-20

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


# URL of the first page
o_url = "https://ugcnet.nta.nic.in/documents/page/"
page_number = 1
max_pages = 3
base_url = 'http://it.delhigovt.nic.in'


def extract_pdf_links(url, base):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    pdf_links = [link.get('href') for link in links if link.get('href') and (link.get('href').endswith('.pdf') or link.get('href').endswith('.PDF'))]
    print(pdf_links[0])
    return pdf_links

all_pdf_links = []
while True:
    url = f"{o_url}{page_number}"
    print(f"Fetching {url}")

    pdf_links = extract_pdf_links(url, base_url)
    all_pdf_links.extend(pdf_links)
    response = requests.get(url)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.content, 'html.parser')
    if page_number == max_pages:
        break

    page_number += 1


df = pd.DataFrame(list(set(all_pdf_links)), columns=['PDF Links'])
df.to_csv('pdf_links.csv', index=False)
print(df)

Fetching https://ugcnet.nta.nic.in/documents/page/1
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/10/2023102789.pdf
Fetching https://ugcnet.nta.nic.in/documents/page/2
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2023/06/2023060820.pdf
Fetching https://ugcnet.nta.nic.in/documents/page/3
https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68dc6014898c309e86bf/uploads/2018/04/2018071864.pdf
                                            PDF Links
0   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
1   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
2   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
3   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
4   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
5   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
6   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
7   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
8   https://cdnbbsr.s3waas.gov.in/s301eee509ee2f68...
9   https://cdnb

# Ministry of Finance

In [ ]:
link = "https://doe.gov.in/orders-circulars"
base_link = "https://doe.gov.in/"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup, base_url):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(urljoin(base_url, link['href']))
          pdf_links.append(urljoin(base_url, link['href']))
        else:
          print(urljoin(base_url, link['href']), 'not a pdf')
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


def find_and_follow_links(soup, visited_urls):
    nested_links = []
    # print(base_link)
    for link in soup.find_all('a', href=True):
        href = link['href']
        if 'orders-circulars' in href:
          # print(href)
          if href.startswith('http') or href.startswith('www'):
              full_url = href
          else:
              full_url = base_link + href
          # print(full_url)

          if full_url not in visited_urls:
              nested_links.append(full_url)
    return nested_links

# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup, base_link))

    nested_links = find_and_follow_links(soup, visited_urls)
    # print('nested Links ;', nested_links)

    for nested_link in list(set(nested_links)):
        nested_link = nested_link+'?page='
        page_number = 0
        # print(nested_link)
        while True:
          print('nested_page :', nested_link+str(page_number))
          soup = fetch_page(nested_link+str(page_number))
          if not soup or soup is None:
              print('Soup Not GOOD')
              pass
          pdfs = extract_pdf_links(soup, base_link)
          page_number+=1
          pdf_links.update(pdfs)
          a_tag = soup.find('a', title='Go to next page')
          if not a_tag:
              break

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)

https://doe.gov.in/#main-content not a pdf
https://www.india.gov.in/hi not a pdf
https://www.india.gov.in/ not a pdf
https://doe.gov.in/hi not a pdf
https://doe.gov.in/ not a pdf
https://doe.gov.in/#content not a pdf
https://doe.gov.in/screen-reader-access not a pdf
https://doe.gov.in/hi/orders-circulars not a pdf
https://doe.gov.in/ not a pdf
https://doe.gov.in/home not a pdf
https://www.g20.org/en not a pdf
https://swachhbharat.mygov.in/ not a pdf
https://www.makeinindia.com/home not a pdf
https://doe.gov.in/ not a pdf
https://doe.gov.in/about-department not a pdf
https://doe.gov.in/profiles-of-ministers not a pdf
https://doe.gov.in/organization-structure not a pdf
https://doe.gov.in/about-department not a pdf
https://doe.gov.in/files/inline-documents/DoE_Allocation_Business.pdf
https://doe.gov.in/list-of-financial-advisers not a pdf
https://doe.gov.in/whos-who not a pdf
https://doe.gov.in/ not a pdf
https://doe.gov.in/central-pay-commission not a pdf
https://doe.gov.in/order-central

KeyboardInterrupt: 

# Tamil government

In [ ]:
link = "https://www.tn.gov.in/go_view/dept"
base_link = "https://www.tn.gov.in/"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout, verify = False)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup, base_url):
    # links = soup.find_all('a', href=True)
    # pdf_links = []
    # for link in links:
    #   try:
    #     if '.pdf' in link['href'] or '.PDF' in link['href']:
    #       print(link['href'])
    #       pdf_links.append(link['href'])
    #     # else:
    #     #   print(urljoin(base_url, link['href']), 'not a pdf')
    #   except:
    #     print('Problem IN the PARSING')
    #     pass
    # return pdf_links

    script_tags = soup.find_all('script')

    # Regular expression to find URLs ending with .pdf in the script tags
    pdf_url_pattern = re.compile(r'http[s]?://[^\s]+\.pdf')

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Find all PDF URLs in the script content
            pdf_urls = pdf_url_pattern.findall(script_content)
            for pdf_url in pdf_urls:
                # download_pdf(pdf_url, download_path)
                print(pdf_url)
    return pdf_urls


def find_and_follow_links(soup, visited_urls):
    nested_links = []
    # print(base_link)
    for link in soup.find_all('a', href=True):
        href = link['href']
        # print(href)
        if href.startswith('http') or href.startswith('www'):
            full_url = href
        else:
            full_url = base_link + href
        # print(full_url)

        if full_url not in visited_urls:
            nested_links.append(full_url)
    return nested_links

# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    # pdf_links.update(extract_pdf_links(soup, base_link))

    nested_links = find_and_follow_links(soup, visited_urls)
    # print('nested Links ;', nested_links)

    for nested_link in list(set(nested_links)):
        nested_link = nested_link+'?page='
        page_number = 0
        # print(nested_link)
        while True:
          print('nested_page :', nested_link+str(page_number))
          soup = fetch_page(nested_link+str(page_number))
          if not soup or soup is None:
              print('Soup Not GOOD')
              pass
          pdfs = extract_pdf_links(soup, base_link)
          page_number+=1

          pdf_links.update(pdfs)
          a_tag = soup.find('a', string = 'next')
          if not a_tag:
              break

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


nested_page : https://www.tn.gov.in//go_view/dept/7?page=0


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


nested_page : https://www.tn.gov.in/ta/go_view/dept/?page=0


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


nested_page : https://www.tn.gov.in//go_view/dept/22?page=1


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


nested_page : https://www.tn.gov.in//go_view/dept/22?page=2


KeyboardInterrupt: 

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import os

# URL of the website containing department links
base_url = "https://www.tn.gov.in/go_view/dept"

# Function to extract and download PDF links from a page
def extract_and_download_pdfs(url, download_path):
    response = requests.get(url, verify = False)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all script tags
    script_tags = soup.find_all('script')

    # Regular expression to find URLs ending with .pdf in the script tags
    pdf_url_pattern = re.compile(r'http[s]?://[^\s]+\.pdf')

    for script in script_tags:
        script_content = script.string
        if script_content:
            # Find all PDF URLs in the script content
            pdf_urls = pdf_url_pattern.findall(script_content)
            for pdf_url in pdf_urls:
                # download_pdf(pdf_url, download_path)
                print(pdf_url)

# Function to download a PDF
def download_pdf(url, download_path):
    response = requests.get(url)
    pdf_name = os.path.join(download_path, url.split('/')[-1])

    with open(pdf_name, 'wb') as pdf_file:
        pdf_file.write(response.content)
    print(f'Downloaded: {pdf_name}')

# Function to scrape department links and extract PDFs
def scrape_departments(base_url, download_path):
    response = requests.get(base_url, verify = False)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Assuming department links are in <a> tags with class 'department-link'
    department_links = soup.find_all('a', class_='department-link')

    for link in department_links:
        print(link)
        department_url = link['href']
        if not department_url.startswith('http'):
            department_url = base_url + department_url

        extract_and_download_pdfs(department_url, download_path)

# Main execution
download_path = './pdfs'
os.makedirs(download_path, exist_ok=True)
scrape_departments(base_url, download_path)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tn.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# Directorate of Education , maharashtra

In [ ]:
link = "https://dte.maharashtra.gov.in/government-resolutions-orders-letters-circulars-e"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout, verify = False)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if link['href'].endswith('.pdf') or link['href'].endswith('.PDF'):
          print(link['href'])
          pdf_links.append(link['href'])
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


# Function to recursively extract PDFs from a website
def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    pdf_links.update(extract_pdf_links(soup))

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])

# Save the DataFrame to a CSV file
df.to_csv('all_pdf_links.csv', index=False)

# Display the DataFrame
print(df)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dte.maharashtra.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'dte.maharashtra.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://gr.maharashtra.gov.in/Site/Upload/Government%20Resolutions/Marathi/202405091651500908.pdf
https://dte.maharashtra.gov.in/wp-content/uploads/2023/10/Selection-GR-13_10_2023_-202310131100027108.pdf
https://dte.maharashtra.gov.in/wp-content/uploads/2023/08/FS-Increase-intake-GR.pdf
https://dte.maharashtra.gov.in/wp-content/uploads/2023/07/GR-Unaided-30-June-2023.pdf
https://gr.maharashtra.gov.in/Site/Upload/Government%20Resolutions/Marathi/202305301536057008.pdf
https://dte.maharashtra.gov.in/wp-content/uploads/2023/04/परदेशी-schol_4th-Oct.-2018.pdf
https://gr.maharashtra.gov.in/Site/Upload/Government%20Resolutions/Marathi/202303271151151908.pdf
https://gr.maharashtra.gov.in/Site/Upload/Government%20Resolutions/Marathi/202302151559052208......pdf
https://dte.maharashtra.gov.in/wp-content/uploads/2022/10/Scholarship-Award-GR-03-10-2022.pdf
https://dte.maharashtra.gov.in/wp-content/uploads/2022/06/Har-Ghar-Zanda-GR-dtd-20.06.2022.pdf
https://gr.maharashtra.gov.in/Site/Upload/Governm

# Directorate of Town Planning, Maharashtra

In [ ]:
base_url = 'https://dtp.maharashtra.gov.in/en/notice-and-notification-circular?page='
page_number = 0
max_pages = 72
pdf_url = ''

def extract_pdf_links(url, base):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(link['href'])
          pdf_links.append(link['href'])
      except:
        print('Problem IN the PARSING')
        pass
    # pdf_links = [base+link.get('href') for link in links if ('.pdf' in link.get('href') or '.PDF' in link.get('href'))]
    print(len(pdf_links))
    return pdf_links

all_pdf_links = []
while True:
    url = f"{base_url}{page_number}"
    print(f"Fetching {url}")

    pdf_links = extract_pdf_links(url, pdf_url)
    all_pdf_links.extend(pdf_links)
    response = requests.get(url)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.content, 'html.parser')
    if page_number == max_pages:
        break

    page_number += 1

Fetching https://dtp.maharashtra.gov.in/en/notice-and-notification-circular?page=0
Problem IN the PARSING
https://dtp.maharashtra.gov.in/sites/default/files/Notification/DTP/Download_2.pdf
https://dtp.maharashtra.gov.in/sites/default/files/Notification/DTP/Download_3.pdf
https://dtp.maharashtra.gov.in/sites/default/files/Notification/UDP_DTP/Download.pdf 
https://dtp.maharashtra.gov.in/sites/default/files/Notification/UDP_DTP/13%20b.pdf 
https://dtp.maharashtra.gov.in/sites/default/files/Notification/UDP_DTP/12%20b_0.pdf 
https://dtp.maharashtra.gov.in/sites/default/files/Notification/DTP/Download_0.pdf
https://dtp.maharashtra.gov.in/sites/default/files/Notification/UDP_DTP/11.b.pdf 
https://dtp.maharashtra.gov.in/sites/default/files/Notification/UDP_DTP/9.b.pdf 
https://dtp.maharashtra.gov.in/sites/default/files/Notification/UDP_DTP/10.b.pdf 
https://dtp.maharashtra.gov.in/sites/default/files/Notification/UDP_DTP/7.b.pdf 
Problem IN the PARSING
10
Fetching https://dtp.maharashtra.gov.

In [ ]:
df = pd.DataFrame(list(set(all_pdf_links)), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

                                             PDF Links
0    https://dtp.maharashtra.gov.in/sites/default/f...
1    https://dtp.maharashtra.gov.in/sites/default/f...
2    https://dtp.maharashtra.gov.in/sites/default/f...
3    https://dtp.maharashtra.gov.in/sites/default/f...
4    https://dtp.maharashtra.gov.in/sites/default/f...
..                                                 ...
714  https://dtp.maharashtra.gov.in/sites/default/f...
715  https://dtp.maharashtra.gov.in/sites/default/f...
716  https://dtp.maharashtra.gov.in/sites/default/f...
717  https://dtp.maharashtra.gov.in/sites/default/f...
718  https://dtp.maharashtra.gov.in/sites/default/f...

[719 rows x 1 columns]


# Directorate of Accounts and Treasuries

In [ ]:
link = "https://mahakosh.maharashtra.gov.in/index.php/en/circulars-a-orders/85"
base_link = "https://mahakosh.maharashtra.gov.in/"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout, verify = False)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup, base_url):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(urljoin(base_url, link['href']))
          pdf_links.append(urljoin(base_url, link['href']))
        # else:
        #   print(urljoin(base_url, link['href']), 'not a pdf')
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


def find_and_follow_links(soup, visited_urls):
    nested_links = []
    # print(base_link)
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('http') or href.startswith('www'):
            full_url = href
        else:
            full_url = base_link + href
        # print(full_url)

        if full_url not in visited_urls:
            nested_links.append(full_url)
    return nested_links

def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    # pdf_links.update(extract_pdf_links(soup, base_link))

    nested_links = find_and_follow_links(soup, visited_urls)
    # print('nested Links ;', nested_links)

    for nested_link in list(set(nested_links)):
        # print(nested_link)
        print('nested_page :', nested_link)
        soup = fetch_page(nested_link)
        if not soup or soup is None:
            print('Soup Not GOOD')
            continue
        pdfs = extract_pdf_links(soup, base_link)
        pdf_links.update(pdfs)

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mahakosh.maharashtra.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


nested_page : https://mahakosh.maharashtra.gov.in//index.php/en/circulars-a-orders/105


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mahakosh.maharashtra.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://mahakosh.maharashtra.gov.in/images/stories/upload/2024/Vehicle_Advance_Circular_for_Recovery220420241713724200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2024/wagh_solewad_masram_exam030420241712082600.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2024/sharad_jadhav030420241712082600.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/श्री060620231685989800.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/Smt_thakur220220231677004200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/श्रीमती_देशपांडे_दि220220231677004200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/श्री220220231677004200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/श्री220220231677004200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/श्री220220231677004200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/01012023_circular020120231672597800.pdf
http

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mahakosh.maharashtra.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

https://mahakosh.maharashtra.gov.in/images/stories/upload/2024/Vehicle_Advance_Circular_for_Recovery220420241713724200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/E-tender_Notice240720231690137000.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/Auction_Notice200620231687199400.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2024/Tender_Notice_2024_compressed080520241715106600.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2024/नोटीस_MH_15_AA_7086240120241706034600.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/वहान_विक्री_नोटीस_new221220231703183400.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/Darpatrak_suchna_12191220231702924200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/Cleaning_Tender_2024191220231702924200.pdf
https://mahakosh.maharashtra.gov.in/images/stories/upload/2023/VAHAN_VIKRI-2-5291120231701196200.pdf
https://mahakosh.maharashtra.gov.in/i

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mahakosh.maharashtra.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


nested_page : http://validator.w3.org/check?uri=https://mahakosh.maharashtra.gov.in/
HTTP error occurred: 403 Client Error: Forbidden for url: https://validator.w3.org/check?uri=https://mahakosh.maharashtra.gov.in/ - http://validator.w3.org/check?uri=https://mahakosh.maharashtra.gov.in/
Soup Not GOOD
nested_page : http://jigsaw.w3.org/css-validator/validator?uri=https%3A%2F%2Fmahakosh.maharashtra.gov.in%2F&profile=css3&usermedium=all&warning=1&vextwarning=


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'validator.w3.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'jigsaw.w3.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


                                              PDF Links
0     https://mahakosh.maharashtra.gov.in/images/sto...
1     https://mahakosh.maharashtra.gov.in/images/sto...
2     https://mahakosh.maharashtra.gov.in/images/sto...
3     https://mahakosh.maharashtra.gov.in/images/sto...
4     https://mahakosh.maharashtra.gov.in/images/sto...
...                                                 ...
1320  https://mahakosh.maharashtra.gov.in/images/sto...
1321  https://mahakosh.maharashtra.gov.in/images/sto...
1322  https://mahakosh.maharashtra.gov.in/images/sto...
1323  https://mahakosh.maharashtra.gov.in/images/sto...
1324  https://mahakosh.maharashtra.gov.in/images/sto...

[1325 rows x 1 columns]


# Chief Secretory Office , Haryana


In [ ]:
base_url = 'https://csharyana.gov.in/en-us/Categories/cid/4?page='
page_number = 1
max_pages = 57
pdf_url = 'https://csharyana.gov.in/'

def extract_pdf_links(url, base):
    response = requests.get(url, verify = False)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(base+link['href'])
          pdf_links.append(base+link['href'])
      except:
        print('Problem IN the PARSING')
        pass
    # pdf_links = [base+link.get('href') for link in links if ('.pdf' in link.get('href') or '.PDF' in link.get('href'))]
    print(len(pdf_links))
    return pdf_links

all_pdf_links = []
while True:
    url = f"{base_url}{page_number}"
    print(f"Fetching {url}")

    pdf_links = extract_pdf_links(url, pdf_url)
    all_pdf_links.extend(pdf_links)
    response = requests.get(url, verify = False)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.content, 'html.parser')
    if page_number == max_pages:
        break

    page_number += 1

Fetching https://csharyana.gov.in/en-us/Categories/cid/4?page=1


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'csharyana.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Problem IN the PARSING
Problem IN the PARSING
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Services-I/14541.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Grievances/14538.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Services-I/14524.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Grievances/14509.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Central-Committee-of-Examinations/14454.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Monitoring-and-Coordination/14447.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Human-Resources-I/14505.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Services-I/14415.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Cabinet-Section/14391.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-Orders/Cabinet-Section/14390.pdf
https://csharyana.gov.in//WriteReadData/Notifications-&-O

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'csharyana.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching https://csharyana.gov.in/en-us/Categories/cid/4?page=2


...

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'csharyana.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching https://csharyana.gov.in/en-us/Categories/cid/4?page=57


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'csharyana.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Problem IN the PARSING
Problem IN the PARSING
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/Vigilance%20I%20&%20Vigilance%20II/2983.pdf
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/Vigilance%20I%20&%20Vigilance%20II/3012.pdf
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/Political%20and%20Parliamentary%20Affairs/3890.pdf
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/Political%20and%20Parliamentary%20Affairs/4078.pdf
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/Establishment-I/4431.pdf
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/Cabinet%20Section/5238.pdf
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/Cabinet%20Section/5239.pdf
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/Cabinet%20Section/5923.pdf
https://csharyana.gov.in//WriteReadData/Notifications%20&%20Orders/General%20Services-I/5858.pdf
https://csharyana.gov.in/

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'csharyana.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
df = pd.DataFrame(list(set(all_pdf_links)), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

                                              PDF Links
0     https://csharyana.gov.in//WriteReadData/Notifi...
1     https://csharyana.gov.in//WriteReadData/Notifi...
2     https://csharyana.gov.in//WriteReadData/Notifi...
3     https://csharyana.gov.in//WriteReadData/Notifi...
4     https://csharyana.gov.in//WriteReadData/Notifi...
...                                                 ...
1129  https://csharyana.gov.in//WriteReadData/Notifi...
1130  https://csharyana.gov.in//WriteReadData/Notifi...
1131  https://csharyana.gov.in//writereaddata/Notifi...
1132  https://csharyana.gov.in//WriteReadData/Notifi...
1133  https://csharyana.gov.in//WriteReadData/Notifi...

[1134 rows x 1 columns]


# Government of West Bengal

In [ ]:
link = "https://wbxpress.com/circulars/"
base_link = ""

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=10):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout, verify = False)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup, base_url):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(urljoin(base_url, link['href']))
          pdf_links.append(urljoin(base_url, link['href']))
        # else:
        #   print(urljoin(base_url, link['href']), 'not a pdf')
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


def find_and_follow_links(soup, visited_urls):
    nested_links = []
    # print(base_link)
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('http') or href.startswith('www'):
            full_url = href
        else:
            full_url = base_link + href
        # print(full_url)

        if full_url not in visited_urls:
            nested_links.append(full_url)
    return nested_links

visited_urls = set()
visited_urls.add(link)
soup = fetch_page(link)
pdf_links = set()
# pdf_links.update(extract_pdf_links(soup, base_link))
nested_links = find_and_follow_links(soup, visited_urls)

for nested_link in list(set(nested_links)):
    if 'circulars' in nested_link:
      print('nested_page :', nested_link)
      soup = fetch_page(nested_link)
      if not soup or soup is None:
          print('Soup Not GOOD')
          continue
      pdfs = extract_pdf_links(soup, base_link)
      pdf_links.update(pdfs)
      pdf_folders = find_and_follow_links(soup, visited_urls)
      for pdf_link in pdf_folders:
        soup = fetch_page(pdf_link)
        if not soup or soup is None:
          print('Soup Not GOOD')
          continue
        pdfs = extract_pdf_links(soup, base_link)
      pdf_links.update(pdfs)

    # return pdf_links

# all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(pdf_links), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbxpress.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


nested_page : https://wbxpress.com/circulars/information-cultural-affairs/


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbxpress.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://wbxpress.com/files/2022/05/2031-ICA.pdf
https://wbxpress.com/files/2012/10/ICA-121.pdf


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbxpress.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbxpress.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbxpress.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: Inse

...

https://wbxpress.com/files/2013/07/2126-LL.pdf


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbxpress.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://wbxpress.com/files/2013/01/Land-Allotment-Policy.pdf


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wbxpress.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(list(pdf_links), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

                                             PDF Links
0       https://wbxpress.com/files/2012/08/592-Edn.pdf
1       https://wbxpress.com/files/2017/01/ED-1009.pdf
2       https://wbxpress.com/files/2019/01/288-Edn.pdf
3      https://wbxpress.com/files/2022/05/2031-ICA.pdf
4       https://wbxpress.com/files/2012/08/802-Edn.pdf
..                                                 ...
146      https://wbxpress.com/files/2012/08/61-Edn.pdf
147  https://finance.wb.gov.in/writereaddata/1091-F...
148     https://wbxpress.com/files/2018/07/987-Edn.pdf
149  https://wbxpress.com/files/2013/06/324-Edn-14-...
150      https://wbxpress.com/files/2012/08/46-Edn.pdf

[151 rows x 1 columns]


# state election commisison gujarat

In [ ]:
link = "https://sec.gujarat.gov.in/imp-imp-jud.htm"
base_link = "https://sec.gujarat.gov.in/"

def fetch_page(url, retries=3, backoff_factor=0.3, timeout=30):
    try:
        retry_strategy = Retry(
            total=retries,
            status_forcelist=[429, 500, 502, 503, 504],
            backoff_factor=backoff_factor
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        http = requests.Session()
        http.mount("http://", adapter)
        http.mount("https://", adapter)

        response = http.get(url, timeout=timeout, verify = False)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx, 5xx)
        return BeautifulSoup(response.content, 'html.parser')

    except requests.Timeout:
        print(f"Request to {url} timed out.")
    except requests.ConnectionError:
        print(f"Failed to connect to {url}.")
    except requests.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err} - {url}")
    except requests.RequestException as req_err:
        print(f"Error fetching page {url}: {req_err}")

    return None


def extract_pdf_links(soup, base_url):
    links = soup.find_all('a', href=True)
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(urljoin(base_url, link['href']))
          pdf_links.append(urljoin(base_url, link['href']))
        # else:
        #   print(urljoin(base_url, link['href']), 'not a pdf')
      except:
        print('Problem IN the PARSING')
        pass
    return pdf_links


def find_and_follow_links(soup, visited_urls):
    nested_links = []
    # print(base_link)
    for link in soup.find_all('a', href=True):
        href = link['href']
        if href.startswith('http') or href.startswith('www'):
            full_url = href
        else:
            full_url = base_link + href
        # print(full_url)

        if full_url not in visited_urls:
            nested_links.append(full_url)
    return nested_links

def extract_pdfs_from_website(url, visited_urls=None):
    if visited_urls is None:
        visited_urls = set()

    if url in visited_urls:
        return []

    visited_urls.add(url)
    soup = fetch_page(url)
    if not soup:
        return []

    pdf_links = set()

    # pdf_links.update(extract_pdf_links(soup, base_link))

    nested_links = find_and_follow_links(soup, visited_urls)
    # print('nested Links ;', nested_links)

    for nested_link in list(set(nested_links)):
        # print(nested_link)
        print('nested_page :', nested_link)
        soup = fetch_page(nested_link)
        if not soup or soup is None:
            print('Soup Not GOOD')
            continue
        pdfs = extract_pdf_links(soup, base_link)
        pdf_links.update(pdfs)

    return pdf_links

all_pdfs = extract_pdfs_from_website(link)
df = pd.DataFrame(list(all_pdfs), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

Request to https://sec.gujarat.gov.in/imp-imp-jud.htm timed out.
Empty DataFrame
Columns: [PDF Links]
Index: []


# finance Dept, Odisha

In [ ]:
base_url = 'https://finance.odisha.gov.in/publication/act-and-rules/compilation-circulars?page='
page_number = 0
max_pages = 1
pdf_url = 'https://finance.odisha.gov.in/'

def extract_pdf_links(url, base):
    response = requests.get(url, verify = False)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(base+link['href'])
          pdf_links.append(base+link['href'])
      except:
        print('Problem IN the PARSING')
        pass
    # pdf_links = [base+link.get('href') for link in links if ('.pdf' in link.get('href') or '.PDF' in link.get('href'))]
    print(len(pdf_links))
    return pdf_links

all_pdf_links = []
while True:
    url = f"{base_url}{page_number}"
    print(f"Fetching {url}")

    pdf_links = extract_pdf_links(url, pdf_url)
    all_pdf_links.extend(pdf_links)
    response = requests.get(url, verify = False)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.content, 'html.parser')
    if page_number == max_pages:
        break

    page_number += 1

df = pd.DataFrame(list(set(all_pdf_links)), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

Fetching https://finance.odisha.gov.in/publication/act-and-rules/compilation-circulars?page=0


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Problem IN the PARSING
https://finance.odisha.gov.in//sites/default/files/2023-07/Compilation%20of%20Circulars_2022-23.pdf
https://finance.odisha.gov.in//sites/default/files/2022-06/Compilation%20of%20Circulars%2C%202021-22.pdf
https://finance.odisha.gov.in//sites/default/files/2021-07/Compilation%20of%20Circular%20Report%20for%202020-21.pdf
https://finance.odisha.gov.in//sites/default/files/2020-10/CIRCULARS_2019-20.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/CIRCULARS_2018-19.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/2017-18-CIRCULARS.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/2016-17-CIRCULARS.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/2015-16-CIRCULARS.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/2014-15-CIRCULARS.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/2013-14-COMPILATION_OF_CIRCULARS.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/2012-13-COMPILA

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching https://finance.odisha.gov.in/publication/act-and-rules/compilation-circulars?page=1


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Problem IN the PARSING
https://finance.odisha.gov.in//sites/default/files/2020-02/CIC_OIFD_02_03.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/2001-02.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1998-99_1999_2000.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1997-98.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1996-97.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1995-96_compressed.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1994-95.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1993-94.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1992-93.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1991-92.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1990-91.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1989-90.pdf
https://finance.odisha.gov.in//sites/default/files/2020-02/1988-89_compressed.pdf
https://finance

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'finance.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


                                            PDF Links
0   https://finance.odisha.gov.in//sites/default/f...
1   https://finance.odisha.gov.in//sites/default/f...
2   https://finance.odisha.gov.in//sites/default/f...
3   https://finance.odisha.gov.in//sites/default/f...
4   https://finance.odisha.gov.in//sites/default/f...
5   https://finance.odisha.gov.in//sites/default/f...
6   https://finance.odisha.gov.in//sites/default/f...
7   https://finance.odisha.gov.in//sites/default/f...
8   https://finance.odisha.gov.in//sites/default/f...
9   https://finance.odisha.gov.in//sites/default/f...
10  https://finance.odisha.gov.in//sites/default/f...
11  https://finance.odisha.gov.in//sites/default/f...
12  https://finance.odisha.gov.in//sites/default/f...
13  https://finance.odisha.gov.in//sites/default/f...
14  https://finance.odisha.gov.in//sites/default/f...
15  https://finance.odisha.gov.in//sites/default/f...
16  https://finance.odisha.gov.in//sites/default/f...
17  https://finance.odisha.g

# Department of General Administration & Grievance, odisha

In [ ]:
base_url = 'https://ga.odisha.gov.in/service-condition/instruction-circulars-guidelines?page='
page_number = 0
max_pages = 4
pdf_url = 'https://ga.odisha.gov.in/'

def extract_pdf_links(url, base):
    response = requests.get(url, verify = False)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(base+link['href'])
          pdf_links.append(base+link['href'])
      except:
        print('Problem IN the PARSING')
        pass
    # pdf_links = [base+link.get('href') for link in links if ('.pdf' in link.get('href') or '.PDF' in link.get('href'))]
    print(len(pdf_links))
    return pdf_links

all_pdf_links = []
while True:
    url = f"{base_url}{page_number}"
    print(f"Fetching {url}")

    pdf_links = extract_pdf_links(url, pdf_url)
    all_pdf_links.extend(pdf_links)
    response = requests.get(url, verify = False)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.content, 'html.parser')
    if page_number == max_pages:
        break

    page_number += 1

df = pd.DataFrame(list(set(all_pdf_links)), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

Fetching https://ga.odisha.gov.in/service-condition/instruction-circulars-guidelines?page=0


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ga.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://ga.odisha.gov.in/https://ga.odisha.gov.in/sites/default/files/2024-05/ListofIAS.pdf
https://ga.odisha.gov.in//sites/default/files/2022-05/01.01.2020.PDF
https://ga.odisha.gov.in/https://ga.odisha.gov.in/sites/default/files/2024-05/ListofIAS.pdf
https://ga.odisha.gov.in//sites/default/files/2022-05/01.01.2020.PDF
Problem IN the PARSING
https://ga.odisha.gov.in//sites/default/files/2024-05/14024%20dated%2008.05.2024%20Filing%20of%20SLPs%20in%20matters%20pertaining%20to%20Odisha%20Civil%20Services%20%28Rehabilitation%20Assistance%29%20Rules%2C%202020_0.pdf
https://ga.odisha.gov.in//sites/default/files/2024-05/13844%20dated%2006.05.2024%20Actions%20to%20be%20taken%20for%20filing%20of%20appeals%20SLP%20regarding..pdf
https://ga.odisha.gov.in//sites/default/files/2024-05/13502%20Reference%20to%20GAPGD%20on%20matters%20relating%20to%20Service%20Conditions.pdf
https://ga.odisha.gov.in//sites/default/files/2024-05/Draft%20supplementary%20agreement%20letter%20no.13262%20dated%2001.05.202

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ga.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching https://ga.odisha.gov.in/service-condition/instruction-circulars-guidelines?page=1


...

Fetching https://ga.odisha.gov.in/service-condition/instruction-circulars-guidelines?page=4


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ga.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://ga.odisha.gov.in/https://ga.odisha.gov.in/sites/default/files/2024-05/ListofIAS.pdf
https://ga.odisha.gov.in//sites/default/files/2022-05/01.01.2020.PDF
https://ga.odisha.gov.in/https://ga.odisha.gov.in/sites/default/files/2024-05/ListofIAS.pdf
https://ga.odisha.gov.in//sites/default/files/2022-05/01.01.2020.PDF
Problem IN the PARSING
https://ga.odisha.gov.in//sites/default/files/2022-02/YP%20Rules_compressed.pdf
https://ga.odisha.gov.in//sites/default/files/2022-02/24300.Grant%20of%20permission%20to%20Government%20servants%20for%20travelling%20outside%20India_compressed.pdf
https://ga.odisha.gov.in//sites/default/files/2022-02/30023.pdf
https://ga.odisha.gov.in//sites/default/files/2022-02/21736.pdf
https://ga.odisha.gov.in//sites/default/files/2022-02/10350.%20Induction%20of%20eminent%20sports%20persons%20in%20Group-B%20Posts%20and%20Services%20under%20Government%20organisations%20and%20PSUs.pdf
https://ga.odisha.gov.in//sites/default/files/2022-02/33044.pdf
https://ga.odisha

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ga.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


                                             PDF Links
0    https://ga.odisha.gov.in//sites/default/files/...
1    https://ga.odisha.gov.in//sites/default/files/...
2    https://ga.odisha.gov.in//sites/default/files/...
3    https://ga.odisha.gov.in//sites/default/files/...
4    https://ga.odisha.gov.in//sites/default/files/...
..                                                 ...
139  https://ga.odisha.gov.in//sites/default/files/...
140  https://ga.odisha.gov.in//sites/default/files/...
141  https://ga.odisha.gov.in//sites/default/files/...
142  https://ga.odisha.gov.in//sites/default/files/...
143  https://ga.odisha.gov.in//sites/default/files/...

[144 rows x 1 columns]


In [ ]:
base_url = 'https://urban.odisha.gov.in/notification/circulars-%26-office-order?page='
page_number = 0
max_pages = 1
pdf_url = 'https://urban.odisha.gov.in/'

def extract_pdf_links(url, base):
    response = requests.get(url, verify = False)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    pdf_links = []
    for link in links:
      try:
        if '.pdf' in link['href'] or '.PDF' in link['href']:
          print(base+link['href'])
          pdf_links.append(base+link['href'])
      except:
        print('Problem IN the PARSING')
        pass
    # pdf_links = [base+link.get('href') for link in links if ('.pdf' in link.get('href') or '.PDF' in link.get('href'))]
    print(len(pdf_links))
    return pdf_links

all_pdf_links = []
while True:
    url = f"{base_url}{page_number}"
    print(f"Fetching {url}")

    pdf_links = extract_pdf_links(url, pdf_url)
    all_pdf_links.extend(pdf_links)
    response = requests.get(url, verify = False)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.content, 'html.parser')
    if page_number == max_pages:
        break

    page_number += 1

df = pd.DataFrame(list(set(all_pdf_links)), columns=['PDF Links'])
df.to_csv('all_pdf_links.csv', index=False)
print(df)

Fetching https://urban.odisha.gov.in/notification/circulars-%26-office-order?page=0


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urban.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://urban.odisha.gov.in//sites/default/files/2023-09/Urban-water-supply_1.pdf
https://urban.odisha.gov.in/https://urban.odisha.gov.in/sites/default/files/2023-03/Drink-From-Tap-Mission.pdf
https://urban.odisha.gov.in//sites/default/files/2023-10/100HOUSEHOLDCONNECTION.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/AMRUT.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/Decentralisedsolidwastemanagement.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/FSSM.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/GARIMA.pdf
https://urban.odisha.gov.in/https://urban.odisha.gov.in/sites/default/files/2023-09/JAGA%20MISSION.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/AWAS.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/MUKTA.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/NULM.pdf
https://urban.odisha.gov.in//sites/default/files/2023-10/AMAPOKHARI-12thOct.pdf
https://urban.odisha.gov.in//sites/default/fi

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urban.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Fetching https://urban.odisha.gov.in/notification/circulars-%26-office-order?page=1


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urban.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


https://urban.odisha.gov.in//sites/default/files/2023-09/Urban-water-supply_1.pdf
https://urban.odisha.gov.in/https://urban.odisha.gov.in/sites/default/files/2023-03/Drink-From-Tap-Mission.pdf
https://urban.odisha.gov.in//sites/default/files/2023-10/100HOUSEHOLDCONNECTION.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/AMRUT.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/Decentralisedsolidwastemanagement.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/FSSM.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/GARIMA.pdf
https://urban.odisha.gov.in/https://urban.odisha.gov.in/sites/default/files/2023-09/JAGA%20MISSION.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/AWAS.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/MUKTA.pdf
https://urban.odisha.gov.in//sites/default/files/2023-09/NULM.pdf
https://urban.odisha.gov.in//sites/default/files/2023-10/AMAPOKHARI-12thOct.pdf
https://urban.odisha.gov.in//sites/default/fi

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'urban.odisha.gov.in'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


                                            PDF Links
0   https://urban.odisha.gov.in//sites/default/fil...
1   https://urban.odisha.gov.in//sites/default/fil...
2   https://urban.odisha.gov.in//sites/default/fil...
3   https://urban.odisha.gov.in//sites/default/fil...
4   https://urban.odisha.gov.in//sites/default/fil...
5   https://urban.odisha.gov.in//sites/default/fil...
6   https://urban.odisha.gov.in//sites/default/fil...
7   https://urban.odisha.gov.in//sites/default/fil...
8   https://urban.odisha.gov.in//sites/default/fil...
9   https://urban.odisha.gov.in/https://urban.odis...
10  https://urban.odisha.gov.in//sites/default/fil...
11  https://urban.odisha.gov.in//sites/default/fil...
12  https://urban.odisha.gov.in//sites/default/fil...
13  https://urban.odisha.gov.in//sites/default/fil...
14  https://urban.odisha.gov.in/https://urban.odis...
15  https://urban.odisha.gov.in//sites/default/fil...
16  https://urban.odisha.gov.in//sites/default/fil...
17  https://urban.odisha.gov